In [ ]:
import numpy as np

In [ ]:
from timeit import default_timer as timer

In [ ]:
NORM_R_TOLERANCE = 1.0e-10

In [ ]:
def lu_solve(A, b):
    pass

In [ ]:
def gs_solve(A, b, n_iter=100):
    pass    

In [ ]:
def cg_solve(A, b, tol=NORM_R_TOLERANCE):
    pass    

In [ ]:
def cg_solve_preconditioned(A, b, p, tol=NORM_R_TOLERANCE):
    pass    

In [ ]:
def test_solvers(A, b):
    start = timer()
    x_lu = lu_solve(A, b)
    stop = timer()
    print(f'LU:\t\t{stop-start:0.5f} s')
    start = timer()
    x_gs = gs_solve(A, b)
    stop = timer()
    print(f'GS:\t\t{stop-start:0.5f} s')
    start = timer()
    x_cg = cg_solve(A, b)
    stop = timer()
    print(f'CG:\t\t{stop-start:0.5f} s')
    return x_lu, x_gs, x_cg

## Example: 2 x 2, integer coefficients

In [ ]:
A = np.array([[4, 1], [1, 3]])
b = np.array([1, 2]).T
x_lu, x_gs, x_cg = test_solvers(A, b)

## Example: 2 x 2, integer coefficients

## Example: spline matrix